# Analysis of the Temporal Evolution of Loads from GCAM-USA

This notebook analyzes the time series of annual total loads from GCAM-USA for the the eleven states in the WECC.

In [ ]:
# Start by importing the packages we need:
import os
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## Suppress Future Warnings


In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)


## Set the Directory Structure

In [ ]:
# Identify the top-level directory and the subdirectory where the data will be stored:
gcam_data_input_dir =  '/Users/burl878/Documents/GODEEEP/Data/TELL_v3/Production_Runs/gcamusa_data/'
data_output_dir =  '/Users/burl878/Documents/Code/code_repos/load_analysis/data/'
image_output_dir =  '/Users/burl878/Documents/Code/code_repos/load_analysis/figures/'


## Process the GridView-Ready Output Files

In [ ]:
# Check to see if the processed output file exist and if not then create it:
if os.path.isfile((os.path.join(data_output_dir, 'GCAM_Rerun_WECC_States_Load_Time_Series.csv'))) == False:

   # Set the scenarios to process:
   scenarios = ['bau_rerun_mc_climate', 'bau_rerun_mc_ira_ccs_climate', 'nz_rerun_mc_ccs_climate', 'nz_rerun_mc_climate', 'nz_rerun_mc_ira_ccs_climate']
    
   #Initiate a counter and empty dataframe to store the results:
   counter = 0
   output_df = pd.DataFrame()

   # Loop over the scenarios and process each time series:
   for s in range(len(scenarios)):
       # Read in the TELL BA output file for that year and scenario:
       gcam_df = pd.read_csv(gcam_data_input_dir + 'GODEEEP_GCAM_USA_electricity_load_' + scenarios[s] + '.csv')
   
       # Rename a few variables for simplicity:
       gcam_df.rename(columns={'region': 'State', 'x': 'Year', 'value': 'NonTran_Load_EJ', 'transportation_value': 'Tran_Load_EJ'}, inplace=True)
    
       # Only keep the columns that are needed:
       gcam_df = gcam_df[['State', 'Year', 'NonTran_Load_EJ', 'Tran_Load_EJ']].copy()
           
       # Compute the total load by summing the transportation and non-transportation components:
       gcam_df['Load_EJ'] = gcam_df['NonTran_Load_EJ'] + gcam_df['Tran_Load_EJ']
    
       # Set the years to process:
       years_to_process = [1975, 1990, 2005, 2010, 2015, 2020, 2025, 2030, 2035, 2040, 2045, 2050]
              
       ## Loop over the years from the start_year to the end_year:
       for year in years_to_process:
           
           # Subset to just the data for the year being processed:
           gcam_subset_df = gcam_df[gcam_df['Year'] == year].copy() 
        
           # Subset the data to just states in the WECC:
           wecc_df = gcam_subset_df[gcam_subset_df['State'].isin(['WA', 'OR', 'CA', 'ID', 'UT', 'NV', 'AZ', 'NM', 'MT', 'WY', 'CO'])]
    
           # Iterate the counter by one:
           counter = counter + 1
            
           # Put the output in a new dataframe:
           output_df.loc[counter, 'Scenario'] = scenarios[s]
           output_df.loc[counter, 'Year'] = year
           output_df.loc[counter, 'Total_Load_TWh'] = ((wecc_df['Load_EJ'].sum().round(6)) * 277.7777778).round(3)
           output_df.loc[counter, 'NonTran_Load_TWh'] = ((wecc_df['NonTran_Load_EJ'].sum().round(6)) * 277.7777778).round(3)
           output_df.loc[counter, 'Tran_Load_TWh'] = ((wecc_df['Tran_Load_EJ'].sum().round(6)) * 277.7777778).round(3)
            
           # Clean up
           del gcam_subset_df, wecc_df
         
   # Write out the dataframe to a .csv file:
   output_df.to_csv((os.path.join(data_output_dir, 'GCAM_Rerun_WECC_States_Load_Time_Series.csv')), sep=',', index=False)

else:
   # Read in the already processed output file:
   output_df = pd.read_csv((os.path.join(data_output_dir, 'GCAM_Rerun_WECC_States_Load_Time_Series.csv')))            
                  
# Preview the future dataframe:
output_df


In [ ]:
# Define a function to plot the time series of annual demand for the WECC:
def plot_wecc_load_time_series(data_input_dir: str, image_output_dir: str, image_resolution: int, save_images=False):
    
    # Read in the output file:
    output_df = pd.read_csv((os.path.join(data_output_dir, 'GCAM_Rerun_WECC_States_Load_Time_Series.csv')))
    
    # Subset to the time series for each interconnection and scenario:
    wecc_historic = output_df[output_df['Scenario'].isin(['bau_rerun_mc_climate'])].loc[(output_df['Year'] <= 2015)]
    wecc_bau_climate = output_df[output_df['Scenario'].isin(['bau_rerun_mc_climate'])].loc[(output_df['Year'] > 2015)]
    wecc_bau_ira_ccs_climate = output_df[output_df['Scenario'].isin(['bau_rerun_mc_ira_ccs_climate'])].loc[(output_df['Year'] > 2015)]
    wecc_nz_ccs_climate = output_df[output_df['Scenario'].isin(['nz_rerun_mc_ccs_climate'])].loc[(output_df['Year'] > 2015)]
    wecc_nz_climate = output_df[output_df['Scenario'].isin(['nz_rerun_mc_climate'])].loc[(output_df['Year'] > 2015)]
    wecc_nz_ira_ccs_climate = output_df[output_df['Scenario'].isin(['nz_rerun_mc_ira_ccs_climate'])].loc[(output_df['Year'] > 2015)]
    
    # Make the raw load plots:
    plt.figure(figsize=(24, 20))
    plt.rcParams['font.size'] = 18
    
    plt.subplot(211)
    #plt.plot(wecc_historic['Year'], wecc_historic['Total_Load_TWh'], color='k', linestyle='-', label='historic', linewidth=3)
    #plt.scatter(wecc_historic['Year'], wecc_historic['Total_Load_TWh'], s=50, c='k', label='')
    plt.plot(wecc_bau_climate['Year'], wecc_bau_climate['Total_Load_TWh'], color='blue', linestyle='-', label='bau_rerun_mc_climate', linewidth=3)
    plt.scatter(wecc_bau_climate['Year'], wecc_bau_climate['Total_Load_TWh'], s=50, c='blue', label='')
    plt.plot(wecc_bau_ira_ccs_climate['Year'], wecc_bau_ira_ccs_climate['Total_Load_TWh'], color='orange', linestyle='-', label='bau_rerun_mc_ira_ccs_climate', linewidth=3)
    plt.scatter(wecc_bau_ira_ccs_climate['Year'], wecc_bau_ira_ccs_climate['Total_Load_TWh'], s=50, c='orange', label='')
    plt.plot(wecc_nz_ccs_climate['Year'], wecc_nz_ccs_climate['Total_Load_TWh'], color='green', linestyle='-', label='nz_rerun_mc_ccs_climate', linewidth=3)
    plt.scatter(wecc_nz_ccs_climate['Year'], wecc_nz_ccs_climate['Total_Load_TWh'], s=50, c='green', label='')
    plt.plot(wecc_nz_climate['Year'], wecc_nz_climate['Total_Load_TWh'], color='red', linestyle='-', label='nz_rerun_mc_climate', linewidth=3)
    plt.scatter(wecc_nz_climate['Year'], wecc_nz_climate['Total_Load_TWh'], s=50, c='red', label='')
    plt.plot(wecc_nz_ira_ccs_climate['Year'], wecc_nz_ira_ccs_climate['Total_Load_TWh'], color='purple', linestyle='-', label='nz_rerun_mc_ira_ccs_climate', linewidth=3)
    plt.scatter(wecc_nz_ira_ccs_climate['Year'], wecc_nz_ira_ccs_climate['Total_Load_TWh'], s=50, c='purple', label='')
    plt.xticks([2005, 2010, 2015, 2020, 2025, 2030, 2035, 2040, 2045, 2050],['2005', '2010', '2015', '2020', '2025','2030','2035','2040','2045','2050'])
    plt.xlim([2019, 2051]); 
    plt.ylabel('Annual Total Load [TWh]')
    plt.grid(True)
    plt.legend(loc='upper left', prop={'size': 18})
    plt.title('Western States Annual Total Load Projections')
    plt.title('a)', loc='left', fontsize=15)
    
    plt.subplot(212)
    #plt.plot(wecc_historic['Year'], wecc_historic['Tran_Load_TWh'], color='k', linestyle='-', label='historic', linewidth=3)
    #plt.scatter(wecc_historic['Year'], wecc_historic['Tran_Load_TWh'], s=50, c='k', label='')
    plt.plot(wecc_bau_climate['Year'], wecc_bau_climate['Tran_Load_TWh'], color='blue', linestyle='-', label='bau_rerun_mc_climate', linewidth=3)
    plt.scatter(wecc_bau_climate['Year'], wecc_bau_climate['Tran_Load_TWh'], s=50, c='blue', label='')
    plt.plot(wecc_bau_ira_ccs_climate['Year'], wecc_bau_ira_ccs_climate['Tran_Load_TWh'], color='orange', linestyle='-', label='bau_rerun_mc_ira_ccs_climate', linewidth=3)
    plt.scatter(wecc_bau_ira_ccs_climate['Year'], wecc_bau_ira_ccs_climate['Tran_Load_TWh'], s=50, c='orange', label='')
    plt.plot(wecc_nz_ccs_climate['Year'], wecc_nz_ccs_climate['Tran_Load_TWh'], color='green', linestyle='-', label='nz_rerun_mc_ccs_climate', linewidth=3)
    plt.scatter(wecc_nz_ccs_climate['Year'], wecc_nz_ccs_climate['Tran_Load_TWh'], s=50, c='green', label='')
    plt.plot(wecc_nz_climate['Year'], wecc_nz_climate['Tran_Load_TWh'], color='red', linestyle='-', label='nz_rerun_mc_climate', linewidth=3)
    plt.scatter(wecc_nz_climate['Year'], wecc_nz_climate['Tran_Load_TWh'], s=50, c='red', label='')
    plt.plot(wecc_nz_ira_ccs_climate['Year'], wecc_nz_ira_ccs_climate['Tran_Load_TWh'], color='purple', linestyle='-', label='nz_rerun_mc_ira_ccs_climate', linewidth=3)
    plt.scatter(wecc_nz_ira_ccs_climate['Year'], wecc_nz_ira_ccs_climate['Tran_Load_TWh'], s=50, c='purple', label='')
    plt.xticks([2005, 2010, 2015, 2020, 2025, 2030, 2035, 2040, 2045, 2050],['2005', '2010', '2015', '2020', '2025','2030','2035','2040','2045','2050'])
    plt.xlim([2019, 2051]); 
    plt.ylabel('Annual Transportation Load [TWh]')
    plt.grid(True)
    plt.legend(loc='upper left', prop={'size': 18})
    plt.title('Western States Annual Transportation Load Projections')
    plt.title('b)', loc='left', fontsize=15)
    
    # If the "save_images" flag is set to true then save the plot to a .png file:
    if save_images == True:
       filename = ('WECC_GCAM_Rerun_Load_Projections.png')
       plt.savefig(os.path.join(image_output_dir, filename), dpi=image_resolution, bbox_inches='tight', facecolor='white')
       plt.close()
    

In [ ]:
plot_wecc_load_time_series(data_input_dir = data_output_dir, 
                           image_output_dir = image_output_dir, 
                           image_resolution = 300, 
                           save_images = True)
